In [30]:
import geopandas as gpd
from geopandas import GeoDataFrame, GeoSeries
import calendar
from pandas import DataFrame
import pandas as pd
import glob

import warnings
warnings.filterwarnings(action='ignore')

# fields names
ped_level= 'ped_level'
import os
general_path = os.path.dirname(os.path.abspath(os.getcwd()))

In [32]:
ml_df = pd.read_csv('output/model_data_final.csv').drop(columns=['Unnamed: 0'])
ml_df

,day part,season,day,buildings,businesses,educationa,Health_ser,Leisure_am,Playground,Sport_faci,synagogues,bus_statio,lighting,trees,bike_trail,parks,density,geometry,ped_level
0,0,2,0,12.580000,0.000000,8,1,0,7,3,4,0.030481,0.060961,0.000000,0,12,3.3,"LINESTRING (184322.70500000002 668574.483, 184...",1
1,1,2,0,12.580000,0.000000,8,1,0,7,3,4,0.030481,0.060961,0.000000,0,12,3.3,"LINESTRING (184322.70500000002 668574.483, 184...",0
2,2,2,0,12.580000,0.000000,8,1,0,7,3,4,0.030481,0.060961,0.000000,0,12,3.3,"LINESTRING (184322.70500000002 668574.483, 184...",2
3,3,2,0,12.580000,0.000000,8,1,0,7,3,4,0.030481,0.060961,0.000000,0,12,3.3,"LINESTRING (184322.70500000002 668574.483, 184...",0
4,0,2,1,12.580000,0.000000,8,1,0,7,3,4,0.030481,0.060961,0.000000,0,12,3.3,"LINESTRING (184322.70500000002 668574.483, 184...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473795,3,0,0,16.378889,0.001021,4,2,0,2,7,0,0.006125,0.042872,0.103097,1,6,1.4,"LINESTRING (180665.74800000002 670989.905, 180...",4
473796,0,0,1,16.378889,0.001021,4,2,0,2,7,0,0.006125,0.042872,0.103097,1,6,1.4,"LINESTRING (180665.74800000002 670989.905, 180...",4
473797,1,0,1,16.378889,0.001021,4,2,0,2,7,0,0.006125,0.042872,0.103097,1,6,1.4,"LINESTRING (180665.74800000002 670989.905, 180...",4
473798,2,0,1,16.378889,0.001021,4,2,0,2,7,0,0.006125,0.042872,0.103097,1,6,1.4,"LINESTRING (180665.74800000002 670989.905, 180...",4


In [33]:



from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier

from sklearn.pipeline import Pipeline

data_feature = ml_df.drop(columns=[ped_level,'geometry'])
x_tree = data_feature.to_numpy()
y_tree = ml_df[ped_level].to_numpy()
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score




In [34]:
# This code run on the four seasons separately
# Variables that won't change during the loop

my_pipeline = Pipeline([
    ('classifier', DecisionTreeClassifier())
])
cv = ShuffleSplit(n_splits=10, test_size=0.3)

param_grid = {'classifier__criterion': ["gini", "entropy","log_loss"],"classifier__splitter": ['best', 'random'],'classifier__max_features':['auto','sqrt','log2',None]}

# Divide the data to training and test
X_train, X_test, y_train, y_test = train_test_split(x_tree, y_tree, test_size=0.2, random_state=0)
# Find the optimal parameters and cross validation
search =GridSearchCV(my_pipeline, param_grid=param_grid,cv=cv,n_jobs=-1)
search.fit(X_train, y_train)
# model accuracy and performance
y_pred =search.best_estimator_.predict(X_test)

cm= confusion_matrix(y_test,y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print("Confusion Matrix:")
print(cm)
print("Best accuracy:", search.best_score_)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# feature importance
best_params = list(search.best_params_.values())
dt = DecisionTreeClassifier(random_state=0,criterion= best_params[0], max_features=best_params[1], splitter=best_params[2]).fit(x_tree, y_tree)
new_df = DataFrame(data={'feature_importance':dt.feature_importances_.round(3)*100,'features_name':data_feature.columns}).sort_values('feature_importance',ascending=0)
print(new_df)


Confusion Matrix:
[[13518  5550  2344   615   117]
 [ 6292  6295  4232  1317   255]
 [ 2647  4633  6966  4002   721]
 [  767  1596  4385  7142  3121]
 [  145   252   863  3421 13564]]
Best accuracy: 0.4919920500914591
Accuracy: 0.5011080624736176
Precision: 0.5006743952461294
Recall: 0.5011080624736176
F1 Score: 0.5007463405439272
    feature_importance features_name
1                 22.8        season
0                 20.2      day part
3                  9.1     buildings
12                 6.6      lighting
15                 5.4         parks
16                 4.6       density
2                  4.2           day
13                 4.0         trees
5                  3.7    educationa
10                 3.5    synagogues
4                  3.5    businesses
9                  3.2    Sport_faci
6                  2.8    Health_ser
8                  2.6    Playground
7                  1.9    Leisure_am
11                 1.2    bus_statio
14                 0.7    bike_trail
